In [1]:
import os
import pandas as pd

In [33]:
# Create a function to read all files in a folder and store them as tables in a dictionary

def create_tables_from_files(data_folder, column_names=None):
    
    tables = {}  # Initialize an empty dictionary to store tables

    # Check if the data folder exists
    if not os.path.exists(data_folder):
        raise FileNotFoundError(f"The data folder '{data_folder}' does not exist.")

    # List all files in the data folder
    files = os.listdir(data_folder)

    for file_name in files:
        file_path = os.path.join(data_folder, file_name)

        # Check if the file is a CSV file (you can modify this for other file formats)
        if file_name.endswith(".csv"):
            try:
                # Read the CSV file into a DataFrame
                table = pd.read_csv(file_path, names=column_names)

                # Store the DataFrame in the tables dictionary with the file name as the key
                tables[file_name] = table

                print(f"Table created for {file_name}")
            except Exception as e:
                print(f"Error reading {file_name}: {str(e)}")
        elif file_name.endswith(".xlsx"):
            try:
                # Read the Excel file into a DataFrame
                table = pd.read_excel(file_path)

                # Store the DataFrame in the tables dictionary with the file name as the key
                tables[file_name] = table

                print(f"Table created for {file_name}")
            except Exception as e:
                print(f"Error reading {file_name}: {str(e)}")

    return tables


column_names = ['unnamed', 'user_id', 'funnel_stage', 'timestamp', 'profit', 'os']
data_folder = "data"  # Specify the path to your data folder
data_tables = create_tables_from_files(data_folder, column_names)


Table created for banner.xlsx
Table created for native.csv
Table created for organic_traffic.csv
Table created for other.csv
Table created for partner_network.csv
Table created for rich.xlsx
Table created for search.csv
Table created for video.csv


In [34]:
data_tables2 = data_tables.copy()  # Make a copy of the dictionary to avoid modifying the original dictionary

# Number of rows & columns
for table_name, table in data_tables2.items():
    table["ad_channel"] = table_name.split('.')[0]  # Add a new column to identify the source table
    print(f"Number of rows & columns in {table_name}: {table.shape}")

Number of rows & columns in banner.xlsx: (576169, 6)
Number of rows & columns in native.csv: (117575, 7)
Number of rows & columns in organic_traffic.csv: (531584, 7)
Number of rows & columns in other.csv: (22588, 7)
Number of rows & columns in partner_network.csv: (387430, 7)
Number of rows & columns in rich.xlsx: (64088, 6)
Number of rows & columns in search.csv: (1491205, 7)
Number of rows & columns in video.csv: (655068, 7)


In [35]:
# Delete first column if number of columns is 7
for table_name, table in data_tables2.items():
    if table.shape[1] == 7:
        table.drop(columns=["unnamed"], inplace=True)
        print(f"Deleted first column in {table_name}")

Deleted first column in native.csv
Deleted first column in organic_traffic.csv
Deleted first column in other.csv
Deleted first column in partner_network.csv
Deleted first column in search.csv
Deleted first column in video.csv


In [38]:
# Merge all dataframes
data = pd.concat(data_tables2.values(), axis=0)

# Lowercase all values in funnel_stage column
data['funnel_stage'] = data['funnel_stage'].str.lower()

# replace nan with 0 in profit column
data['profit'] = data['profit'].fillna(0)

# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Sort by timestamp
data.sort_values(by=['timestamp'], inplace=True)

# Reset the index
data.reset_index(drop=True, inplace=True)

data

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,219687,interest,150000,0.0,Android,banner
1,897013,interest,150000,0.0,Android,organic_traffic
2,242700,interest,150000,0.0,Android,organic_traffic
3,143856,interest,150000,0.0,Android,organic_traffic
4,26717,consideration,150000,0.0,Android,organic_traffic
...,...,...,...,...,...,...
997102,929180,intent,224682,0.0,iOS,search
997103,1034498,intent,225430,0.0,iOS,search
997104,947521,consideration,225482,0.0,Android,organic_traffic
997105,720354,purchase,225495,50.0,Android,organic_traffic


In [24]:
adv_info = pd.read_json('data/advertising_information.json')
# replace nan with 0
adv_info = adv_info.fillna(0)
adv_info

,ad_channel,price
0,search,1.50
1,banner,0.50
2,video,0.75
3,native,2.00
4,partner_network,1.00
5,rich,2.50
6,organic_traffic,0.00
7,other,0.00


In [39]:
# Add price column
data = pd.merge(data, adv_info, on='ad_channel', how='left')
data

,user_id,funnel_stage,timestamp,profit,os,ad_channel,price
0,219687,interest,150000,0.0,Android,banner,0.5
1,897013,interest,150000,0.0,Android,organic_traffic,0.0
2,242700,interest,150000,0.0,Android,organic_traffic,0.0
3,143856,interest,150000,0.0,Android,organic_traffic,0.0
4,26717,consideration,150000,0.0,Android,organic_traffic,0.0
...,...,...,...,...,...,...,...
997102,929180,intent,224682,0.0,iOS,search,1.5
997103,1034498,intent,225430,0.0,iOS,search,1.5
997104,947521,consideration,225482,0.0,Android,organic_traffic,0.0
997105,720354,purchase,225495,50.0,Android,organic_traffic,0.0


In [40]:
# Save to csv
data.to_csv('data/advertising_data.csv', index=False)
